In [352]:
import sympy as sm
import numpy as np
from numpy import sin, cos, tan, arcsin, arccos, arctan
from sympy.physics.mechanics import *
import itertools


def findEquivalentRotationAngles(angles, first_rot_order, second_rot_order, test=False):
    t1, t2, t3 = sm.symbols('theta_1, theta_2, theta_3')   # Rotation angles for (first_rot_order)

    N = ReferenceFrame('N')
    B = ReferenceFrame('B')

    B.orient_body_fixed(N, (t1, t2, t3), first_rot_order)
    C = B.dcm(N)    # DCM from N to B

    syms, sym_eqns = constructEquivAngleFormulas(C, second_rot_order)
    alpha_eqn, beta_eqn, gamma_eqn, beta_check_eqn = sym_eqns

    a, b, g = syms

    t_vals = {t1: np.radians(angles[0]),
              t2: np.radians(angles[1]),
              t3: np.radians(angles[2])}
    equiv_a_angle = sm.solve(alpha_eqn.subs(t_vals), a, dict=True)[0]
    equiv_g_angle = sm.solve(gamma_eqn.subs(t_vals), g, dict=True)[0]
    equiv_b_angle = sm.solve(beta_eqn.subs(t_vals), b, dict=True)[0]
    equiv_bc_angle = sm.solve(beta_check_eqn.subs(t_vals).subs(equiv_a_angle).subs(equiv_g_angle), b, dict=True)[0]

    equiv_angles = equiv_a_angle | equiv_bc_angle | equiv_g_angle

    if not test:
        return equiv_angles
    else:
        # Test to see if the resultant rotation matrices are identical (to within 1E-10 precision)
        Bp = ReferenceFrame('Bp')
        Bp.orient_body_fixed(N, (a, b, g), second_rot_order)
        Cp = Bp.dcm(N)
        return (np.array(C.subs(t_vals)).astype(np.float64).round(10) == np.array(Cp.subs(equiv_angles)).astype(np.float64).round(10)).all()


 
def constructEquivAngleFormulas(C, newRotOrder):
    a, b, g = sm.symbols('alpha beta gamma')             # Rotation angles for (second_rot_order) set

    N = ReferenceFrame('N')
    Bp = ReferenceFrame('Bp')   # 'Dummy' B frame used to examine DCM and extract inverse transformation formulas

    Bp.orient_body_fixed(N, (a, b, g), newRotOrder)
    Cp = Bp.dcm(N)  # DCM from N to Bp

    # Construct the proper Euler angle formulas using the rotation order to inform which indicies of the DCM to call
    amounts, rot_order, rot_matrices = Bp._parse_consecutive_rotations((a, b, g), newRotOrder)
    if rot_order[0] == rot_order[2]:
        # Proper Euler angles
        idx1 = rot_order[1]-1
        idx2 = rot_order[0]-1
        idx3 = 5 - (rot_order[0] + rot_order[1])
        
        a_idxs = [(idx2, idx1), (idx2, idx3)]
        b_idxs = (idx2, idx2)
        g_idxs = [(idx1, idx2), (idx3, idx2)]

    else:
        # Tait-Bryan angles
        idx1 = rot_order[2]-1
        idx2 = rot_order[0]-1
        idx3 = 5 - (rot_order[0] + rot_order[2])

        a_idxs = [(idx1, idx3), (idx1, idx1)]
        b_idxs = (idx1, idx2)
        g_idxs = [(idx3, idx2), (idx2, idx2)]

    alpha_eqn = sm.Eq((C[a_idxs[0]]/C[a_idxs[1]]), (Cp[a_idxs[0]]/Cp[a_idxs[1]]).trigsimp())
    beta_eqn = sm.Eq(C[b_idxs], Cp[b_idxs])
    gamma_eqn = sm.Eq(C[g_idxs[0]]/C[g_idxs[1]], (Cp[g_idxs[0]]/Cp[g_idxs[1]]).trigsimp())
    beta_check_eqn = sm.Eq((C[a_idxs[1]]/C[b_idxs]), (Cp[a_idxs[1]]/Cp[b_idxs]).trigsimp())

    syms = [a, b, g]
    sym_eqns = [alpha_eqn, beta_eqn, gamma_eqn, beta_check_eqn]
    return syms, sym_eqns


def test_all_rotation_combinations():
    legal_rots = []

    for i in itertools.product([1,2,3], [1,2,3], [1,2,3]):
        if i[0] != i[1] and i[1] != i[2]:
            rot_str = '{}{}{}'.format(i[0],i[1],i[2])
            legal_rots.append(rot_str)

    for i in itertools.product(legal_rots, legal_rots):
        if i[0] != i[1]:
            print('{} to {}: {}'.format(i[0], i[1], findEquivalentRotationAngles((20,10,-10), i[0], i[1], test=True)))

# findEquivalentRotationAngles((20,10,-10), '321', '313')
test_all_rotation_combinations()

121 to 123: True
121 to 131: True
121 to 132: True
121 to 212: True
121 to 213: True
121 to 231: True
121 to 232: True
121 to 312: True
121 to 313: True
121 to 321: True
121 to 323: True
123 to 121: True
123 to 131: True
123 to 132: True
123 to 212: True
123 to 213: True
123 to 231: True
123 to 232: True
123 to 312: True
123 to 313: True
123 to 321: True
123 to 323: True
131 to 121: True
131 to 123: True
131 to 132: True
131 to 212: True
131 to 213: True
131 to 231: True
131 to 232: True
131 to 312: True
131 to 313: True
131 to 321: True
131 to 323: True
132 to 121: True
132 to 123: True
132 to 131: True
132 to 212: True
132 to 213: True
132 to 231: True
132 to 232: True
132 to 312: True
132 to 313: True
132 to 321: True
132 to 323: True
212 to 121: True
212 to 123: True
212 to 131: True
212 to 132: True
212 to 213: True
212 to 231: True
212 to 232: True
212 to 312: True
212 to 313: True
212 to 321: True
212 to 323: True
213 to 121: True
213 to 123: True
213 to 131: True
213 to 132: Tr